### **Deep Convolutional Neural Network**



In [ ]:
#Initially we download the zipped data

import os 
if not os.path.isfile('data.zip'):
    !wget https://github.com/LeonardoBerti07/Deep-Learning-Algorithms-for-financial-time-serie-modeling-/blob/main/Dataset/data.zip
    print('data downloaded.')
else:
    print('data already existed.')

In [ ]:
# load packages

import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from tqdm import tqdm 
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import torch
from torch.utils import data
import torch.nn as nn
import torch.optim as optim

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


### **Data**
The dataset in the folder Dataset is the LOBSTER dataset zipped and normalized. I have combined the data of the 5 stocks available for free. I used the version with 10 levels, so we have 40 columns, in fact for every level we have a quadruple wiht the ask and bid prices and with the volumes associated, for more information i reference to the thesis. 

I used 70% to do the training, 15% to do the validation and 15% for the testing.

In [ ]:
# please change the data_path to your local path and unzip the file

data_path =  "/data.npy"

dec = np.load(data_path)

train_size = int(0.85 * dec.shape[0])
val_size = int(0.15 * dec.shape[0])

print(dec.shape)
dec_train = dec[:train_size]
dec_val = dec[train_size:]

dec_test = np.load(data_path2)

In [99]:
#Label the data with the method explained in the thesis

def labeling(X, T):

  [N, D] = X.shape
  print(N)
  Y = np.zeros((X.shape[0] - 3*T + 1))
  alpha = 0.00072
  media = []
  for i in range(0, X.shape[0]- 3*T + 1):
    ask_minus = X[i:i+T, :1]
    bid_minus = X[i:i+T, 2:3]
    ask_plus = X[i+T:i+2*T, :1]
    bid_plus = X[i+T:i+2*T, 2:3]
    m_minus = (ask_minus + bid_minus) / 2
    m_minus = np.sum(m_minus) / T
    m_plus = (ask_plus + bid_plus) / 2
    m_plus = np.sum(m_plus) / T
    media.append((m_plus - m_minus) / m_minus)
    if (m_plus - m_minus) / m_minus < -alpha:
      label = 1
    elif (m_plus - m_minus) / m_minus > alpha:
      label = 0
    else:
      label = 2
    Y[i] = label
  
  plt.hist(Y)
  plt.show()

  return Y

In [100]:
class Dataset(data.Dataset):
    """Characterizes a dataset for PyTorch"""
    def __init__(self, x, y, num_classes, dim):
        """Initialization""" 
        self.num_classes = num_classes
        self.dim = dim
        self.x = x   
        self.y = y
       
        self.length = x.shape[0] - 2*T + 1

        x = torch.from_numpy(x)
        self.x = torch.unsqueeze(x, 1)
        self.y = torch.from_numpy(y)

    def __len__(self):
        """Denotes the total number of samples"""
        return self.length

    def __getitem__(self, i):
        input = self.x[i:i+self.dim, :]
        input = input.permute(1, 0, 2)
        
        return input, self.y[i]

In [ ]:
#Hyperparameters

batch_size = 64
epochs = 50
T = 50   #horizon    
lr = 0.00008

y_val = labeling(dec_val, T)
y_test = labeling(dec_test, T)
y_train = labeling(dec_train, T)

dataset_val = Dataset(dec_val, y_val, num_classes=3, dim=50)
dataset_test = Dataset(dec_test, y_test, num_classes=3, dim=50)
dataset_train = Dataset(dec_train, y_train, num_classes=3, dim=50)

train_loader = torch.utils.data.DataLoader(dataset=dataset_train, batch_size=batch_size, shuffle=True)
val_loader = torch.utils.data.DataLoader(dataset=dataset_val, batch_size=batch_size, shuffle=False)
test_loader = torch.utils.data.DataLoader(dataset=dataset_test, batch_size=batch_size, shuffle=False)


### **Model Architecture**
the structure is made up of 15 convolutional layer and 3 fully connected layer. The activation function used is LeakyReLU. As for the hyperparameters, a learning rate equal to 0.00008, 50 epochs and a batch size of 64 was used. The model in total has 168,132,646 parameters.

In [102]:
class CNN(nn.Module):
    def __init__(self, y_len):
        super().__init__()
        self.y_len = y_len
        
        # convolution blocks
        self.conv1 = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=32, kernel_size=(1,2), stride=(1,2)),
            nn.LeakyReLU(negative_slope=0.01),
            nn.BatchNorm2d(32),
            nn.Conv2d(in_channels=32, out_channels=32, kernel_size=(4,1)),
            nn.LeakyReLU(negative_slope=0.01),
            nn.BatchNorm2d(32),
            nn.Conv2d(in_channels=32, out_channels=32, kernel_size=(4,1)),
            nn.LeakyReLU(negative_slope=0.01),
            nn.BatchNorm2d(32),
        )
        self.conv2 = nn.Sequential(
            nn.Conv2d(in_channels=32, out_channels=32, kernel_size=(1,2), stride=(1,2)),
            nn.LeakyReLU(negative_slope=0.01),
            nn.BatchNorm2d(32),
            nn.Conv2d(in_channels=32, out_channels=32, kernel_size=(4,1)),
            nn.LeakyReLU(negative_slope=0.01),
            nn.BatchNorm2d(32),
            nn.Conv2d(in_channels=32, out_channels=32, kernel_size=(4,1)),
            nn.LeakyReLU(negative_slope=0.01),
            nn.BatchNorm2d(32),
        )
        self.conv3 = nn.Sequential(
            nn.Conv2d(in_channels=32, out_channels=32, kernel_size=(1,10)),
            nn.LeakyReLU(negative_slope=0.01),
            nn.BatchNorm2d(32),
            nn.Conv2d(in_channels=32, out_channels=32, kernel_size=(4,1)),
            nn.LeakyReLU(negative_slope=0.01),
            nn.BatchNorm2d(32),
            nn.Conv2d(in_channels=32, out_channels=32, kernel_size=(4,1)),
            nn.LeakyReLU(negative_slope=0.01),
            nn.BatchNorm2d(32),
        )
        self.inp1 = nn.Sequential(
            nn.Conv2d(32, 64, (3,1), padding="same"),
            nn.LeakyReLU(negative_slope=0.01),
            nn.Conv2d(64, 64, (1,1), padding="same"),
            nn.LeakyReLU(negative_slope=0.01),
            #nn.MaxPool2d(2, 2),

            nn.Conv2d(64, 128, (3,1), padding="same"),
            nn.LeakyReLU(negative_slope=0.01),
            nn.Conv2d(128, 128, (1,1), padding="same"),
            nn.LeakyReLU(negative_slope=0.01),

            nn.Conv2d(128, 256, (3,1), padding="same"),
            nn.LeakyReLU(negative_slope=0.01),
            nn.Conv2d(256, 256, (1,1), padding="same"),
            nn.LeakyReLU(negative_slope=0.01),
            nn.Conv2d(256, 256, 1),
            nn.LeakyReLU(negative_slope=0.01),
            
        )
        
        #fully connected layers
        self.linear1 = nn.Linear(20992, 4096)
        self.linear2 = nn.Linear(4096, 4096)
        self.linear3 = nn.Linear(4096, 3)

        #activation
        self.activation = nn.LeakyReLU(negative_slope=0.01)
      
    def forward(self, x):
      #print(x.shape)
      x = self.conv1(x)
      #print(x.shape)
      x = self.conv2(x)
      #print(x.shape)
      x = self.conv3(x)
      #print(x.shape)
      x = self.inp1(x)
       
      x = torch.flatten(x, 1)
      x = self.activation(self.linear1(x))
      #print(x.shape)
      x = self.activation(self.linear2(x))
      #print(x.shape)
      out = self.linear3(x)
      forecast_y = torch.softmax(out, dim=1)
      
      return forecast_y

### **Model Training**

In [103]:
model = CNN(3)
model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr)

def batch_gd(model, criterion, optimizer, train_loader, test_loader, epochs):
    
    train_losses = np.zeros(epochs)
    test_losses = np.zeros(epochs)
    best_test_loss = np.inf
    best_test_epoch = 0

    for it in tqdm(range(epochs)):
        
        model.train()
        t0 = datetime.now()
        train_loss = []
        for inputs, targets in train_loader:
            # move data to GPU
            inputs, targets = inputs.to(device, dtype=torch.float), targets.to(device, dtype=torch.int64)

            # zero the parameter gradients
            optimizer.zero_grad()

            # Forward pass
            outputs = model(inputs)
            loss = criterion(outputs, targets)

            # Backward and optimize
            loss.backward()
            optimizer.step()
            train_loss.append(loss.item())

        # Get train loss and test loss
        train_loss = np.mean(train_loss) 
    
        model.eval()
        test_loss = []
        n_correct = 0.
        n_total = 0.
        for inputs, targets in test_loader:
            inputs, targets = inputs.to(device, dtype=torch.float), targets.to(device, dtype=torch.int64)      
            outputs = model(inputs)
            _, predictions = torch.max(outputs, 1)
            loss = criterion(outputs, targets)
            test_loss.append(loss.item())
            n_correct += (predictions == targets).sum().item()
            n_total += targets.shape[0]

        test_acc = n_correct / n_total
        print(f"Test acc: {test_acc:.4f}")
        test_loss = np.mean(test_loss)

        # Save losses
        train_losses[it] = train_loss
        test_losses[it] = test_loss
        
        #We save the best model
        if test_loss < best_test_loss:
            torch.save(model, '/best_model_CNN')
            best_test_loss = test_loss
            best_test_epoch = it
            print('model saved')

        dt = datetime.now() - t0
        print(f'Epoch {it+1}/{epochs}, Train Loss: {train_loss:.4f}, \
          Validation Loss: {test_loss:.4f}, Duration: {dt}, Best Val Epoch: {best_test_epoch}')
        
    return train_losses, test_losses

In [ ]:
print("------- List Hyper Parameters -------")
print("epochs   ->   " + str(epochs))
print("learningRate   ->   " + str(lr))
print("training range   ->   " + str(train_size))
print("horizon    ->     " + str(T))
print("Optimizer   ->    AdamW")

train_losses, val_losses = batch_gd(model, criterion, optimizer, train_loader, val_loader, epochs)

plt.figure(figsize=(15,6))
plt.plot(train_losses, label='train loss')
plt.plot(val_losses, label='validation loss')
plt.legend()

### **Model Testing**

In [ ]:
model = torch.load('/best_model_CNN')

n_correct = 0.
n_total = 0.
all_targets = []
all_predictions = []

for inputs, targets in test_loader:
    # Move to GPU
    inputs, targets = inputs.to(device, dtype=torch.float), targets.to(device, dtype=torch.int64)

    # Forward pass
    outputs = model(inputs)
    
    # Get prediction
    # torch.max returns both max and argmax
    _, predictions = torch.max(outputs, 1)

    # update counts
    n_correct += (predictions == targets).sum().item()
    n_total += targets.shape[0]

    all_targets.append(targets.cpu().numpy())
    all_predictions.append(predictions.cpu().numpy())

test_acc = n_correct / n_total
print(f"Test acc: {test_acc:.4f}")
  
all_targets = np.concatenate(all_targets)    
all_predictions = np.concatenate(all_predictions)    

In [ ]:
print('accuracy_score:', accuracy_score(all_targets, all_predictions))
print(classification_report(all_targets, all_predictions, digits=4))
print(confusion_matrix(all_targets, all_prediction, normalize=True))